<a href="https://colab.research.google.com/github/EduardoComparato/YouTrue/blob/main/YouTrue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# Instalar Frameworks
!pip install -q google-adk google-genai youtube-transcript-api gradio



In [29]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

# Verifica se a chave de API já está definida no ambiente
# Se não estiver, tenta obter do userdata do Colab
if not os.getenv("GOOGLE_API_KEY"):
    try:
        os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    except Exception as e:
        print(f"Erro ao obter GOOGLE_API_KEY do userdata: {e}")
        print("Por favor, certifique-se de que a chave 'GOOGLE_API_KEY' está definida nos Secrets do Colab.")
        # Se não conseguir obter, o código pode falhar posteriormente ao usar a API

In [30]:
# Configura o cliente da SDK do Gemini
# Adicionado um bloco try-except caso a chave de API não esteja disponível
try:
    from google import genai
    client = genai.Client()
    MODEL_ID = "gemini-2.0-flash"
except Exception as e:
    print(f"Erro ao configurar o cliente Gemini: {e}")
    print("Verifique se sua GOOGLE_API_KEY está correta e acessível.")
    client = None # Define client como None para evitar erros posteriores

In [31]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

In [32]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import gradio as gr # Para criar a interface gráfica

warnings.filterwarnings("ignore")

In [33]:
import uuid
from typing import Optional

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str, user_id: Optional[str] = None, session_id: Optional[str] = None) -> str:
    # Verifica se o cliente Gemini foi inicializado com sucesso
    if client is None:
        return "Erro: Cliente Gemini não inicializado. Verifique sua chave de API."

    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()

    # Gera user_id e session_id se não forem fornecidos
    user_id_to_use = user_id if user_id else str(uuid.uuid4())
    session_id_to_use = session_id if session_id else str(uuid.uuid4())
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id=user_id_to_use, session_id=session_id_to_use)
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    # Mantido a criação de Content e Part conforme o código original do usuário
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
# Itera assincronamente pelos eventos retornados durante a execução do agente
    try:
        for event in runner.run(user_id=user_id_to_use, session_id=session_id_to_use, new_message=content):
            if event.is_final_response():
              if event.content is not None:
                for part in event.content.parts:
                  if part.text is not None:
                   final_response += part.text
                   final_response += "\n"
    except Exception as e:
        return f"Erro durante a execução do agente {agent.name}: {e}"

    return final_response

In [34]:
def to_markdown(text):
  # Função mantida, o output será usado diretamente no componente Gradio Markdown
  text = text.replace('•', '  *')
  return text # Retorna a string formatada em Markdown

In [35]:
##########################################
# --- Transcritor de Vídeos --- #
##########################################
def API_transcritor(ID_digitado):
  try:
    from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound, TranscriptsDisabled

    ytt_api = YouTubeTranscriptApi()
    transcricao = ytt_api.fetch(ID_digitado, languages=['pt-BR', 'en-US', 'pt', 'en','es'])

    return transcricao
  except NoTranscriptFound:
        print(f"No transcript found for video ID: {ID_digitado} in the specified languages or any other language.")
        return None
  except TranscriptsDisabled:
        print(f"Transcripts are disabled for video ID: {ID_digitado}.")
        return None
  except Exception as e:
        print(f"An error occurred while fetching the transcript for video ID {ID_digitado}: {e}")
        return None
  except TranscriptsDisabled:
        print(f"Transcripts are disabled for video ID: {ID_digitado}.")
        return None
  except Exception as e:
        print(f"An error occurred while fetching the transcript for video ID {ID_digitado}: {e}")
        return None

In [36]:
################################################
# --- Agente 1: Informações do vídeo --- #
################################################
# Verifica se a chave de API do YouTube está definida
YOUTUBE_API_KEY = userdata.get("YOUTUBE_API_KEY")
youtube = None # Inicializa youtube como None
if YOUTUBE_API_KEY:
    try:
        from googleapiclient.discovery import build
        youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)
    except Exception as e:
        print(f"Erro ao construir o cliente da API do YouTube: {e}")
        print("Verifique se sua YOUTUBE_API_KEY está correta.")


def agente_informativo(ID_digitado):
    # Verifica se o cliente YouTube foi inicializado com sucesso
    if youtube is None:
        return "Erro: Cliente da API do YouTube não inicializado. Verifique sua chave de API do YouTube."

    try:
        request = youtube.videos().list(
            part="snippet,contentDetails",
            id=ID_digitado
        )
        response = request.execute()

        if response and 'items' in response and response['items']:
            video_info = response['items'][0]
            title = video_info['snippet']['title']
            duration = video_info['contentDetails']['duration']
            thumbnail = video_info['snippet']['thumbnails']['high']['url']

            # Formata as informações para uma string amigável
            info_string = f"Título: {title}\n"
            info_string += f"Duração: {duration}\n"
            info_string += f"Thumbnail: {thumbnail}\n"

            return info_string
        else:
            return "Vídeo não encontrado ou informações indisponíveis."
    except Exception as e:
        return f"Ocorreu um erro ao buscar as informações do vídeo: {e}"

In [37]:
################################################
# --- Agente 2: Revisor de texto --- #
################################################
# Define o agente revisor fora da função principal para que seja criado apenas uma vez
revisor = Agent(
    name="agente_revisor",
    model="gemini-2.0-flash",
    instruction="""
    Você é um revisor de texto. Quero que pegue o texto mais recente fornecido pelo transcritor e retire elementos de minutagem e tópico,
    somente deixando o texto, separando em parágrafos ou não somente conforme for necessário. Caso o texto esteja em outra língua,
    sem ser o português. Traduza para o português.
    """,
    description="Agente que revisa textos",
)

def agente_revisor_call(transcricao):
    if not transcricao or "Erro:" in transcricao: # Não chama o agente se a transcrição for vazia ou contiver erro
        return "Não foi possível revisar o texto devido a um problema na transcrição."
    entrada_do_agente_revisor = f"Texto a ser revisado: {transcricao}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado


In [38]:
######################################
# --- Agente 3: Buscador e Primeiro Verificador --- #
######################################
# Define o agente buscador fora da função principal para que seja criado apenas uma vez
buscador = Agent(
    name="agente_buscador",
    model="gemini-2.0-flash",
    instruction="""
        Você é um buscador de fontes informacionais. A partir do texto fornecido pelo revisor,
        detecte o tema e pontos abordados e procure sobre em, NO MÁXIMO, 20 fontes confiáveis e
        de relevância no tema, onde as informações sejam por texto
        (evitando artigos de opinião e fontes muito imparciais).
        Enfatize, nessas fontes, as partes onde ocorram incongruências entre informações do texto e das fontes.
        Ao final, forneça os links das fontes pesquisadas.
        """,
    description="Agente que busca fontes relacionadas e faz uma primeira comparação",
    tools=[google_search]
)

def agente_buscador_call(texto_revisado):
    if not texto_revisado or "Erro:" in texto_revisado or "Não foi possível revisar" in texto_revisado: # Não chama o agente se o texto revisado for vazio ou contiver erro
         return "Não foi possível realizar a busca e comparação devido a um problema na revisão do texto."
    entrada_do_agente_buscador = f"Texto revisado: {texto_revisado}"
    # Executa o agente
    busca_comparada = call_agent(buscador, entrada_do_agente_buscador)
    return busca_comparada

In [39]:
##########################################
# --- Agente 4: Aponta-erros --- #
##########################################
# Define o agente de erros fora da função principal para que seja criado apenas uma vez
erros = Agent(
    name="agente_erros",
    model="gemini-2.0-flash",
    instruction="""
        Você aponta os erros de uma fonte informacional, tendo como base um texto e uma análise subsequente
        constando links de fontes confiáveis. Quero que vasculhe essas fontes, para confirmar se não há informações erradas no texto original.


        ###


        Com a análise de erros pronta, indique os trechos que estão errados, o porquê de estarem errados, e uma correção para tais trechos.
        Para cada correção, aponte a fonte que está sendo utilizada. Ao fim, indique a porcentagem do quanto o conteúdo do texto original estava
        comprometido por erros de informação.


        ###


        Ao fim, indique a confiabilidade do texto original a partir de 5 níveis, indicados por emojis,
        levando em consideração a porcentagem estabelecida anteriormente. Especifique quais as porcentagens levadas em conta na análise.

        Grau de confiabilidade:

        Confiável (0% a 7% de erros) -> 🟢

        Bom (8% a 20%) -> 🔵

        Mediano (20% a 45%) -> 🟡

        Ruim (46% a 60%) -> 🟠

        Grave (61% a 100%) -> 🔴

        ###

        Caso o texto final esteja em inglês, passe para o português.
        """,
    description="Agente que aponta erros, correções e grau de confiabilidade.",
    tools=[google_search]
)

def agente_erros_call(texto_revisado, busca_comparada):
    if not busca_comparada or "Erro:" in busca_comparada or "Não foi possível realizar a busca" in busca_comparada: # Não chama o agente se a busca for vazia ou contiver erro
        return "Não foi possível analisar os erros devido a um problema na busca e comparação."
    entrada_do_agente_erros = f"Texto original: {texto_revisado}, Buscador: {busca_comparada}"
    # Executa o agente
    texto_corrigido = call_agent(erros, entrada_do_agente_erros)
    return texto_corrigido

In [40]:
# --- Função principal que será chamada pelo Gradio ---
def processar_entrada(tipo_analise, conteudo_entrada):
    """
    Processa a entrada do usuário (ID de vídeo ou texto) usando a cadeia de agentes.

    Args:
        tipo_analise (str): O tipo de análise escolhido ('ID' ou 'Texto Completo').
        conteudo_entrada (str): O ID do vídeo ou o texto completo a ser analisado.

    Retorna:
        tuple: Uma tupla contendo os resultados de cada etapa do processo para exibição no Gradio.
               (info_video_str, transcricao_str, texto_revisado_str, busca_comparada_str, texto_corrigido_str)
    """
    info_video_str = ""
    transcricao_str = ""
    texto_revisado_str = ""
    busca_comparada_str = ""
    texto_corrigido_str = ""

    if not conteudo_entrada:
        return ("Por favor, insira um ID de vídeo ou texto para analisar.", "", "", "", "")

    if tipo_analise == "ID":
        ID_digitado = conteudo_entrada
        info_video_str = agente_informativo(ID_digitado)
        transcricao_str = API_transcritor(ID_digitado)

        if "Erro:" in transcricao_str or "Não foi possível obter a transcrição" in transcricao_str or "Erro inesperado" in transcricao_str:
             texto_revisado_str = "Processamento interrompido devido a erro na transcrição."
             busca_comparada_str = "Processamento interrompido devido a erro na transcrição."
             texto_corrigido_str = "Processamento interrompido devido a erro na transcrição."
        else:
            texto_revisado_str = agente_revisor_call(transcricao_str)
            if "Erro:" in texto_revisado_str or "Não foi possível revisar" in texto_revisado_str:
                 busca_comparada_str = "Processamento interrompido devido a erro na revisão."
                 texto_corrigido_str = "Processamento interrompido devido a erro na revisão."
            else:
                busca_comparada_str = agente_buscador_call(texto_revisado_str)
                if "Erro:" in busca_comparada_str or "Não foi possível realizar a busca" in busca_comparada_str:
                     texto_corrigido_str = "Processamento interrompido devido a erro na busca."
                else:
                     texto_corrigido_str = agente_erros_call(texto_revisado_str, busca_comparada_str)


    elif tipo_analise == "Texto Completo":
        texto_completo = conteudo_entrada
        transcricao_str = texto_completo # Neste caso, a "transcrição" é o texto fornecido

        texto_revisado_str = agente_revisor_call(transcricao_str)
        if "Erro:" in texto_revisado_str or "Não foi possível revisar" in texto_revisado_str:
             busca_comparada_str = "Processamento interrompido devido a erro na revisão."
             texto_corrigido_str = "Processamento interrompido devido a erro na revisão."
        else:
            busca_comparada_str = agente_buscador_call(texto_revisado_str)
            if "Erro:" in busca_comparada_str or "Não foi possível realizar a busca" in busca_comparada_str:
                 texto_corrigido_str = "Processamento interrompido devido a erro na busca."
            else:
                 texto_corrigido_str = agente_erros_call(texto_revisado_str, busca_comparada_str)


    # Aplica a formatação markdown aos resultados que se beneficiam dela
    texto_revisado_str = to_markdown(texto_revisado_str)
    busca_comparada_str = to_markdown(busca_comparada_str)
    texto_corrigido_str = to_markdown(texto_corrigido_str)


    return info_video_str, transcricao_str, texto_revisado_str, busca_comparada_str, texto_corrigido_str

In [41]:
# --- Criação da Interface Gradio ---

# Define os componentes de entrada
input_components = [
    gr.Radio(["ID", "Texto Completo"], label="Tipo de Análise", value="ID"), # Valor padrão é ID
    gr.Textbox(label="Insira o ID do YouTube ou o Texto Completo")
]

# Define os componentes de saída para cada etapa do processo
output_components = [
    gr.Textbox(label="Informações do Vídeo (se ID)", interactive=False),
    gr.Textbox(label="Transcrição Original / Texto Fornecido", interactive=False),
    gr.Markdown(label="Texto Revisado pelo Agente 2"), # Usa Markdown para formatação
    gr.Markdown(label="Busca e Comparação pelo Agente 3"), # Usa Markdown para formatação
    gr.Markdown(label="Análise de Erros e Confiabilidade pelo Agente 4") # Usa Markdown para formatação
]

In [42]:
# Cria a interface Gradio
iface = gr.Interface(
    fn=processar_entrada, # Função Python a ser executada
    inputs=input_components, # Componentes de entrada
    outputs=output_components, # Componentes de saída
    title="YouTrue - Verificador de Informações",
    description="Analise vídeos do YouTube (via ID) ou textos completos para verificar informações usando uma cadeia de Agentes Gemini.",
    allow_flagging="never" # Desabilita a opção de "flagging" padrão do Gradio
)

In [43]:
# Lança a interface Gradio
# No Google Colab, isso irá gerar um link público
iface.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://da849ee0280992e4d8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://da849ee0280992e4d8.gradio.live
